In [2]:
def initialize_shape_parameters(target_features, mean, pcaBasis, pcaVariance, r, t, s):
    """
    Gets a shape estimation.
    
    Args:
      target_features: the features of the input 2D image [x1, y1, x2, y2, ...]       (2*f,1)
      mean: the mean 3D feature points                                                (3*f, 1)
      pcaBasis: the pca basis for the feature points                                  (3*f, c)
      pcaVariance: the pca variance for the feature points                            (c, 1)
      r: the current rotation matrix                                                  (3, 3)
      t: the current translation matrix                                               (2, 1)
      s: the current scale                                                            (1)
      where
        f: the number of features
        c: the number of components
      
    Returns:
      A shape estimate
    """
    
    # get the number of features and components
    num_features = int(len(mean)/3)
    num_components = pcaBasis.shape[1]

    # C and h component data buffers
    C = np.zeros((2*num_features, num_components))
    h = np.zeros((2*num_features, 1))
    
    # initialize C
    for i in range(num_features):
        C[2*i]   = s * (r[0,0] * pcaBasis[3*i,:] + r[0,1] * pcaBasis[3*i+1,:] + r[0,2] * pcaBasis[3*i+2,:])
        C[2*i+1] = s * (r[1,0] * pcaBasis[3*i,:] + r[1,1] * pcaBasis[3*i+1,:] + r[1,2] * pcaBasis[3*i+2,:])
        h[2*i]   = target_features[2*i] - s * (np.matmul(r[0], mean[3*i:3*i+3]) + t[0,0])
        h[2*i+1] = target_features[2*i+1] - s * (np.matmul(r[1], mean[3*i:3*i+3]) + t[1,0])
    
    # return the result
    std = np.sqrt(pcaVariance.reshape(num_components,))   # TODO: pass std not variance to function
    bounds = (-3*std, 3*std)
    return lsq_linear(C, h.flatten(), bounds=bounds)['x'].reshape((num_components,1)) # TODO, constraint